In [ ]:
import findspark

findspark.init('C:/Users/Bruno/anaconda3/Lib/site-packages/Spark')

In [2]:
from __future__ import print_function

from pyspark.ml.regression import LinearRegression

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

if __name__ == "__main__":

    # Create a SparkSession (Note, the config section is only for Windows!)
    spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

    # Load up our data and convert it to the format MLLib expects.
    inputLines = spark.sparkContext.textFile("C:/Users/Bruno/Desktop/Python/Projectos Big Data/Spark/regression.txt")
    data = inputLines.map(lambda x: x.split(",")).map(lambda x: (float(x[0]), Vectors.dense(float(x[1]))))

    # Convert this RDD to a DataFrame
    colNames = ["label", "features"]
    df = data.toDF(colNames)

    # Note, there are lots of cases where you can avoid going from an RDD to a DataFrame.
    # Perhaps you're importing data from a real database. Or you are using structured streaming
    # to get your data.

    # Let's split our data into training data and testing data
    trainTest = df.randomSplit([0.5, 0.5])
    trainingDF = trainTest[0]
    testDF = trainTest[1]

    # Now create our linear regression model
    lir = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

    # Train the model using our training data
    model = lir.fit(trainingDF)

    # Now see if we can predict values in our test data.
    # Generate predictions using our linear regression model for all features in our
    # test dataframe:
    fullPredictions = model.transform(testDF).cache()

    # Extract the predictions and the "known" correct labels.
    predictions = fullPredictions.select("prediction").rdd.map(lambda x: x[0])
    labels = fullPredictions.select("label").rdd.map(lambda x: x[0])

    # Zip them together
    predictionAndLabel = predictions.zip(labels).collect()

    # Print out the predicted and actual values for each point
    for prediction in predictionAndLabel:
      print(prediction)


    # Stop the session
    spark.stop()

(-1.8208688844712315, -2.58)
(-1.6939749946088103, -2.54)
(-1.5529817836505642, -2.27)
(-1.4472368754318796, -2.07)
(-1.3203429855694584, -1.91)
(-1.341491967213195, -1.88)
(-1.1793497746112123, -1.77)
(-1.1652504535153876, -1.66)
(-1.3062436644736337, -1.64)
(-1.221647737898686, -1.61)
(-1.1582007929674751, -1.6)
(-1.1511511324195631, -1.59)
(-1.1723001140632998, -1.58)
(-1.1088531691320893, -1.57)
(-1.1934490957070367, -1.53)
(-1.0383565636529661, -1.47)
(-1.1300021507758262, -1.42)
(-0.939661315982194, -1.39)
(-1.0031082609134048, -1.36)
(-0.8409660683114216, -1.3)
(-1.031306903105054, -1.3)
(-0.8127674261197725, -1.29)
(-1.0454062242008784, -1.29)
(-0.8621150499551586, -1.26)
(-0.939661315982194, -1.25)
(-0.7916184444760357, -1.2)
(-0.7845687839281233, -1.12)
(-0.8762143710509832, -1.11)
(-0.7704694628322988, -1.1)
(-0.798668105023948, -1.08)
(-0.7352211600927372, -1.07)
(-0.8339164077635094, -1.04)
(-0.7634198022843864, -1.03)
(-0.6506252335177896, -0.97)
(-0.685873536257351, -0.9